##### Environment Requirements

In [50]:
!pip install -U langchain-ollama langchain-community faiss-cpu

In [51]:
!sudo apt-get update -qq
!sudo apt-get install -y zstd

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 82 not upgraded.


In [52]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [53]:
!ollama serve > ollama.log 2>&1 &

In [54]:
!ps aux | grep ollama

root        1375  0.0  0.3 2230884 43984 ?       Sl   13:33   0:05 ollama serve
root       29247  2.1  5.7 2783404 760716 ?      Sl   15:28   0:04 /usr/local/bin/ollama runner --ollama-engine --model /root/.ollama/models/blobs/sha256-819c2adf5ce6df2b6bd2ae4ca90d2a69f060afeb438d0c171db57daa02e39c3d --port 40797
root       30654  0.0  0.0  46312  8936 ?        D    15:31   0:00 ollama serve
root       30655  0.0  0.0   7372  3608 ?        S    15:31   0:00 /bin/bash -c ps aux | grep ollama
root       30657  0.0  0.0   6480  2384 ?        S    15:31   0:00 grep ollama


In [55]:
!ollama pull mxbai-embed-large

In [56]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434"   # this  local host is communicating to
)                                       # my local ollama application
                                        # for embedding
test_text = "this  is test, what is transformer attention?"
embedding = embeddings.embed_query(test_text)

print("embedding generated!")
print("Vector length:", len(embedding))          # ~1024 aayegi
print("first 5 values (sample):", embedding[:5])

embedding generated!
Vector length: 1024
first 5 values (sample): [-0.0020972237, 0.029994715, -0.022563037, -0.009765678, -0.022581104]


In [57]:
import os

In [58]:
!pip install --upgrade --force-reinstall \
  youtube-transcript-api \
  langchain-community \
  faiss-cpu \
  tiktoken \
  python-dotenv \
  langchain-ollama


  Using cached youtube_transcript_api-1.2.4-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached faiss_cpu-1.13.2-cp310-abi3-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (7.6 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.7 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_ollama-1.0.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_core-1.2.14-py3-none-any.whl.metadata (4.4 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached sqlalchemy-2.0.46-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (9.5 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2

In [59]:
# === Required Imports for Ollama-based YouTube RAG Chatbot ===

# YouTube transcript fetch
from youtube_transcript_api import YouTubeTranscriptApi

# Text splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Ollama Embeddings (local, no API key needed)
from langchain_ollama import OllamaEmbeddings

# Vector Store
from langchain_community.vectorstores import FAISS

# Prompt & LLM
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

# Chain helpers (optional but useful)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("all imports are ready")


all imports are ready


##### Indexing

In [60]:
!pip install youtube-transcript-api==0.6.2

  Using cached youtube_transcript_api-0.6.2-py3-none-any.whl.metadata (15 kB)
Using cached youtube_transcript_api-0.6.2-py3-none-any.whl (24 kB)
  Attempting uninstall: youtube-transcript-api
    Found existing installation: youtube-transcript-api 1.2.4
    Uninstalling youtube-transcript-api-1.2.4:
      Successfully uninstalled youtube-transcript-api-1.2.4


In [61]:
!pip uninstall -y youtube_transcript_api youtube-transcript-api

Found existing installation: youtube-transcript-api 0.6.2
Uninstalling youtube-transcript-api-0.6.2:
  Successfully uninstalled youtube-transcript-api-0.6.2


In [62]:
!pip install youtube-transcript-api==0.6.2

  Using cached youtube_transcript_api-0.6.2-py3-none-any.whl.metadata (15 kB)
Using cached youtube_transcript_api-0.6.2-py3-none-any.whl (24 kB)


In [63]:
pip install --upgrade youtube-transcript-api

  Using cached youtube_transcript_api-1.2.4-py3-none-any.whl.metadata (24 kB)
Using cached youtube_transcript_api-1.2.4-py3-none-any.whl (485 kB)
  Attempting uninstall: youtube-transcript-api
    Found existing installation: youtube-transcript-api 0.6.2
    Uninstalling youtube-transcript-api-0.6.2:
      Successfully uninstalled youtube-transcript-api-0.6.2


In [64]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "LPZh9BOjkQs"

try:
    # Instance banao
    api = YouTubeTranscriptApi()

    # Direct fetch with preferred language
    fetched_transcript = api.fetch(video_id, languages=['en'])  # ya ['en-US'], ['hi-IN'] etc.

    # Ab .text attribute use karo har snippet ka
    Transcript = " ".join(snippet.text for snippet in fetched_transcript)

    print(Transcript)

except Exception as e:
    print("Error bhai:", str(e))

Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the use

In [65]:
Transcript

"Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a hypothetical AI assistant, add on whatever the us

#### step - 2 : indexing-Text Splitting

In [66]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [67]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50)
chunks = splitter.create_documents([Transcript])
documents = chunks

## here chunk size 500 means every chunk
## will holded 500 characters , and chunk overalp
## means in every chunks 50 character will be repeating
## in next chunks..to maintain and retain the context
## of chunks.

In [68]:
len(chunks)

17

In [69]:
chunks[0]

Document(metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over")

In [70]:
chunks[16]

Document(metadata={}, page_content='details of attention and all the other steps in a transformer. Also, on my second channel I just posted a talk I gave a couple months ago about this topic for the company TNG in Munich. Sometimes I actually prefer the content I make as a casual talk rather than a produced video, but I leave it up to you which one of these feels like the better follow-on.')

 ##### Embedding Generation And Storing in Vector Store

In [71]:
from langchain_community.vectorstores import FAISS

In [72]:
# Ollama embeddings use kar rahe hain (local server pe chal raha hai)
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",              # best balanced model (1024 dim)
    base_url="http://127.0.0.1:11434"       # ya "http://localhost:11434" – dono same
)
vectorstore = FAISS.from_documents(chunks, embeddings)
# Test kar lete hain
# test_text = "Bhai ye test hai"
# test_embedding = embeddings.embed_query(test_text)

# print("Embedding ban gaya bhai!")
# print("Embedding length:", len(test_embedding))          # 1024 aayega
# print("Pehle 5 values (sample):", test_embedding[:5])
# print("Success! Ab vector store bana sakta hai.") # ~768 ya 3072 dimensions aayega

In [73]:
vectorstore.index_to_docstore_id

{0: '0390a268-c745-4e2d-9472-8c730af82382',
 1: '6c02d9f2-5458-442d-a527-ee63ed9565a1',
 2: '1483a79a-e30b-4d72-8999-52b78ce2e911',
 3: '64d4005a-5450-4c04-8398-398b34595d0d',
 4: '692b4ae9-7ecd-4031-961e-095f140f8630',
 5: '2f5e65b9-1d6c-46c2-8e1a-baf4060d6d45',
 6: '8061cc1d-d14b-4335-9d64-a2126ae106f2',
 7: 'd29d2659-c103-4b73-bd1a-e308fa3e7689',
 8: '3a1701fd-3c9d-4ebd-8622-e4768aea276b',
 9: 'f9b9c0c3-b86a-4818-a5e6-ee4b86c60f30',
 10: 'bbd65f83-91e9-44a1-b091-b1c28d8e7883',
 11: 'fb4a4d12-5f82-49ed-9c98-2e109930ffd5',
 12: 'a76e1705-abd6-4353-ad8d-3bc6c2585fea',
 13: '23875cd3-9c7a-409c-969c-734e94b310a0',
 14: '180f5879-d6a9-4b4e-8267-12ad0d2ea1cc',
 15: '6989d854-cd20-42b1-928c-ab3fb2d37d90',
 16: '98b3d8f6-f8d5-448e-9154-8d8802c76619'}

### Retriever

In [74]:
retriever = vectorstore.as_retriever(search_type = "similarity" , search_kwarges = {"k":4})

In [75]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7aabd8f15610>, search_kwargs={})

In [76]:
retriever.invoke("what  is large language model")

[Document(id='6c02d9f2-5458-442d-a527-ee63ed9565a1', metadata={}, page_content="answer, and then repeating this over and over with a growing script completing the dialogue. When you interact with a chatbot, this is exactly what's happening. A large language model is a sophisticated mathematical function that predicts what word comes next for any piece of text. Instead of predicting one word with certainty, though, what it does is assign a probability to all possible next words. To build a chatbot, you lay out some text that describes an interaction between a user and a"),
 Document(id='d29d2659-c103-4b73-bd1a-e308fa3e7689', metadata={}, page_content="it's never seen before. Given the huge number of parameters and the enormous amount of training data, the scale of computation involved in training a large language model is mind-boggling. To illustrate, imagine that you could perform one billion additions and multiplications every single second. How long do you think it would take for you

###  Augmentation

In [77]:
from langchain_community.llms import Ollama

In [88]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(
    model="llama2:latest",          # ya llama2:7b
    temperature=0.2
)

In [89]:
prompt = PromptTemplate(
    template = """
    u are helpful assistant.
    answer only from the provided transcript context.
    if the context is sufficient , just say u  dont know.
    {context}
    Question:{question}
  """,
  input_variable = ["context" , "question"]
)

In [90]:
question = "How an LLM knows when to stop an for a prompt output?"
retrived_docs = retriever.invoke(question)

In [91]:
retrived_docs

[Document(id='0390a268-c745-4e2d-9472-8c730af82382', metadata={}, page_content="Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over"),
 Document(id='64d4005a-5450-4c04-8398-398b34595d0d', metadata={}, page_content="this means is even though the model itself is deterministic, a given prompt typically gives a different answer each time it's run. Models learn how to make these predictions by processing an enormous amount of text, typically pulled from the internet. For a standard human to read the amount of text that was used to train GPT-3, for exa

In [82]:
context_text  = "\n\n".join(doc.page_content for doc in retrived_docs)
final_prompt = prompt.format(context = context_text , question = question)
print(final_prompt)


    u are helpful assistant.
    answer only from the provided transcript context.
    if the context is sufficient , just say u  dont know.
    Imagine you happen across a short movie script that describes a scene between a person and their AI assistant. The script has what the person asks the AI, but the AI's response has been torn off. Suppose you also have this powerful magical machine that can take any text and provide a sensible prediction of what word comes next. You could then finish the script by feeding in what you have to the machine, seeing what it would predict to start the AI's answer, and then repeating this over and over

this means is even though the model itself is deterministic, a given prompt typically gives a different answer each time it's run. Models learn how to make these predictions by processing an enormous amount of text, typically pulled from the internet. For a standard human to read the amount of text that was used to train GPT-3, for example, if they re

In [83]:
### Generation

In [84]:
answer = llm.invoke(final_prompt)

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama2`.

In [87]:
pip install -U langchain-ollama

In [92]:
print(llm.invoke("Hello, test message!"))

ResponseError: model 'llama2:latest' not found (status code: 404)